# IMPORTS

In [175]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer 
from sklearn.compose import ColumnTransformer,make_column_selector
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# LOAD DATASET
1. Read the dataset, show its head, shape and description

In [176]:
df = pd.read_csv("https://pokelab.ddns.net/datasets/11_09_23.csv", sep=";")
display(df.head())
display(df.describe())
display(df.info())

,Date,Time,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN,NaN


,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15
count,7765.000000,8991.000000,914.000000,9357.000000,8991.000000,7718.000000,8991.000000,7715.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,0.0,0.0,0.0
mean,2.127521,1099.833166,218.811816,9.688704,939.153376,246.896735,835.493605,113.091251,1456.264598,1022.906128,18.317829,49.234201,1.025530,NaN,NaN,NaN
std,1.463171,217.080037,204.459921,7.559785,266.831429,212.979168,256.817320,48.370108,346.206794,398.484288,8.832116,17.316892,0.403813,NaN,NaN,NaN
min,0.000000,647.000000,7.000000,0.000000,383.000000,2.000000,322.000000,2.000000,551.000000,221.000000,-1.900000,9.200000,0.184700,NaN,NaN,NaN
25%,1.000000,937.000000,67.000000,4.000000,734.500000,98.000000,658.000000,78.000000,1227.000000,731.500000,11.800000,35.800000,0.736800,NaN,NaN,NaN
50%,1.800000,1063.000000,150.000000,7.900000,909.000000,180.000000,806.000000,109.000000,1463.000000,963.000000,17.800000,49.600000,0.995400,NaN,NaN,NaN
75%,2.900000,1231.000000,297.000000,13.600000,1116.000000,326.000000,969.500000,142.000000,1674.000000,1273.500000,24.400000,62.500000,1.313700,NaN,NaN,NaN
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    9357 non-null   object 
 1   Time    9357 non-null   object 
 2   X00     7765 non-null   float64
 3   X01     8991 non-null   float64
 4   X02     914 non-null    float64
 5   X03     9357 non-null   float64
 6   X04     8991 non-null   float64
 7   X05     7718 non-null   float64
 8   X06     8991 non-null   float64
 9   X07     7715 non-null   float64
 10  X08     8991 non-null   float64
 11  X09     8991 non-null   float64
 12  X10     8991 non-null   float64
 13  X11     8991 non-null   float64
 14  X12     8991 non-null   float64
 15  X13     0 non-null      float64
 16  X14     0 non-null      float64
 17  X15     0 non-null      float64
dtypes: float64(16), object(2)
memory usage: 1.3+ MB


None

# DATA PREPROCESSING
2. Eliminate totally null columns and totally null rows, eliminate columns with less than 1/3 of non null values; fill the remaining NaN values with the mean of the column

In [177]:
df.dropna(how="all",inplace=True)
df.dropna(how="all", axis=1,inplace=True)
for col in df.columns:
    
    # removing columns with less then 1/3 of non null values
    if df[col].count()  < len(df[col])/3:
        print(df[col].count())
        print(len(df[col]))
        df.drop(columns=col)
    
    # replacing nan with mean
    if df[col].dtype != "object":
        df[col].fillna(inplace=True,value=df[col].mean())

df.head()
print("number of nulls after procesing {}".format(df.shape[0] - df.dropna().shape[0]))

914
9357
number of nulls after procesing 0


3. Drop Time, convert Date from string to datetime and group by Date using mean as aggregate function

In [178]:
df.drop(columns="Time",inplace=True)
df["Date"]= pd.to_datetime(df["Date"],format="%d/%m/%Y")
df=df.groupby(by="Date").mean()
df.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12
Date,,,,,,,,,,,,,
2004-03-10,1.966667,1316.500000,86.500000,8.450000,912.333333,132.000000,1167.333333,108.833333,1545.500000,1096.000000,12.033333,54.900000,0.765633
2004-03-11,2.234480,1244.166667,104.500000,7.979167,851.958333,148.662364,1277.250000,100.420469,1522.833333,885.250000,9.837500,64.075000,0.775767
2004-03-12,2.748127,1281.666667,158.950492,12.129167,1008.291667,179.824728,1101.875000,116.007604,1627.291667,1084.375000,11.287500,51.095833,0.663104
2004-03-13,2.671980,1330.666667,139.250000,10.916667,992.833333,187.037364,993.208333,118.628802,1595.791667,1245.916667,12.866667,51.533333,0.732296
2004-03-14,2.455313,1361.125000,116.958333,9.637500,943.916667,150.787364,1001.291667,110.503802,1602.375000,1234.208333,16.012500,48.850000,0.849671


4. Preparation of the boolean matrix 

- Discretise continuous values with two bins, kmeans strategy and
onehot-dense encoding
- Discretization/encoding generates 0/1 values; convert the binary
values obtained into boolean, as requested by Apriori

In [179]:
kbd = KBinsDiscretizer(n_bins=2,encode="onehot-dense",strategy='kmeans')
columns=df.columns
df = pd.DataFrame(kbd.fit_transform(df))
df.head()

/home/matteo/datamining/labs/env/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


5. Set the names of two columns generated by the discretisation of each attribute A to A_low, A_high (with discretisation/one-hot-encoding, each original column generates two columns, the first is for the low values, the second for the high values)

In [180]:
new_col=[]

for col in columns:
     new_col.append(col + "_low")
     new_col.append(col + "_high")

df.columns=new_col
df.head()

,X00_low,X00_high,X01_low,X01_high,X02_low,X02_high,X03_low,X03_high,X04_low,X04_high,...,X08_low,X08_high,X09_low,X09_high,X10_low,X10_high,X11_low,X11_high,X12_low,X12_high
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


6. find a value of min_support such that the Apriori algorithm generates at least 8 frequent itemsets with at least 2 items, output the result

In [181]:
# Requirements
min_itemsets = 8
min_item_in_itemset = 2
# "Reasonable" range
support_range = np.arange(1, 0.01, -0.01)

min_support = 0
for s_value in support_range:
    frequent_itemsets = apriori(df, min_support = s_value, use_colnames = True)
    # Calculate the number of itemsets that contain at least `min_item_in_itemset` items
    # frequent_itemsets must contains al least min_itemsets itemsset , and every item set must contains al least min_item_in_itemset item
    itemsets_above_threshold = sum([len(itemset) >= min_item_in_itemset for itemset in frequent_itemsets.itemsets])
    if itemsets_above_threshold >= min_itemsets:
        min_support = s_value
        break
if min_support == 0:
    print("No itemset found! Try again with a bigger range!")
else:
    print(f"I've selected min_support = {min_support:.2f}, which produced␣ , {len(frequent_itemsets)} itemsets, {itemsets_above_threshold} of which had more than {min_item_in_itemset} items")

I've selected min_support = 0.59, which produced␣ , 17 itemsets, 9 of which had more than 2 items


/home/matteo/datamining/labs/env/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/matteo/datamining/labs/env/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/matteo/datamining/labs/env/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/matteo/datamining/labs/env/lib/python3.11/site-packages/mlxtend/frequent_pat

7. find the minimum metric threshold such that at least 100 association rules are extracted from the frequent itemsets found and show the metrics used and the best 10 rules by descending confidence and support

In [182]:
# Requirment
metric_threshold_range = np.arange(20, 0.01, -0.01)
min_association_rule = 10
min_metric_threshold = 0
association_rule_found = 0
current_metric = "confidence"

for metric_value in metric_threshold_range:

    rules = association_rules(frequent_itemsets, metric=current_metric, min_threshold=metric_value)
    if rules.shape[0] >= min_association_rule:
        association_rule_found = rules.shape[0]
        min_metric_threshold = metric_value
        break

if association_rule_found == 0:
    print("No association rule! Try again with a bigger range!")
else:
    print(f"I've selected metric {current_metric}  with metric_value = {metric_value:.2f}, which produced , {association_rule_found} association_rules")
    display(rules.sort_values(by=["confidence","support"],ascending=False).head(n=10))

I've selected metric lift  with metric_value = 1.23, which produced , 12 association_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(X00_low),(X05_low),0.639386,0.734015,0.595908,0.932000,1.269728,0.126589,3.911539,0.589079
5,"(X00_low, X02_low)",(X05_low),0.636829,0.734015,0.593350,0.931727,1.269356,0.125908,3.895893,0.584295
7,(X00_low),"(X05_low, X02_low)",0.639386,0.708440,0.593350,0.928000,1.309921,0.140384,4.049446,0.656090
3,(X07_low),(X05_low),0.685422,0.734015,0.621483,0.906716,1.235283,0.118373,2.851355,0.605474
9,"(X07_low, X02_low)",(X05_low),0.664962,0.734015,0.601023,0.903846,1.231372,0.112931,2.766240,0.560825
11,(X07_low),"(X05_low, X02_low)",0.685422,0.708440,0.601023,0.876866,1.237742,0.115443,2.367821,0.610586
8,"(X05_low, X02_low)",(X07_low),0.708440,0.685422,0.601023,0.848375,1.237742,0.115443,2.074717,0.658791
2,(X05_low),(X07_low),0.734015,0.685422,0.621483,0.846690,1.235283,0.118373,2.051907,0.716089
4,"(X05_low, X02_low)",(X00_low),0.708440,0.639386,0.593350,0.837545,1.309921,0.140384,2.219778,0.811479
10,(X05_low),"(X07_low, X02_low)",0.734015,0.664962,0.601023,0.818815,1.231372,0.112931,1.849154,0.706424
